In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib qt


# Flag to run tests and visualizations for each function 
doRunTests = True


# define the files to be used

xdf_fullFile = "/Users/denismottet/Documents/GitHub/NeuArm-DataAnalysis/data/AgePie/015_AgePie_20211112_1_r(1).xdf"
s_file = "/Users/denismottet/Documents/GitHub/NeuArm-DataAnalysis/data/AgePie/AgePie_A16.snirf"

xdf_fullFile = "/Users/denismottet/Documents/GitHub/NeuArm-DataAnalysis/data/ReArm.lnk/twoTestPatientsForOXY4/C1P07_20210802_1_r.xdf"
s_file = "/Users/denismottet/Documents/GitHub/NeuArm-DataAnalysis/data/ReArm.lnk/twoTestPatientsForOXY4/C1P07_20210802_1_r.snirf"



# Translation of an xdf file to snirf format 

We have the same nirs data in two formats: xdf and snirf. 

The xdf format is a general format for storing time series data (https://github.com/sccn/xdf). 

The snirf format is a format for storing nirs data (https://github.com/fNIRS/snirf).



## Load the xdf file and return only the NIRS and Event streams

In [ ]:
import pyxdf


def get_NIRS_and_Event_streams(x_file):
    """
    Load the xdf file and returns only the NIRS and Event streams
    """
    # load only the NIRS and Event streams
    data, header = pyxdf.load_xdf(
        filename=x_file,
        select_streams=[{"type": "NIRS"}, {"type": "Event"}],
        synchronize_clocks=True,
        dejitter_timestamps=False,
        verbose=False,
    )
    # find the nirs stream among the list of streams
    for i in range(len(data)):
        if data[i]["info"]["type"][0] == "NIRS":
            nirsStream = data[i]
            break
    # find the Event stream among the list of streams
    for i in range(len(data)):
        if data[i]["info"]["type"][0] == "Event":
            eventStream = data[i]
            break
    return nirsStream, eventStream


if doRunTests:
    # load the xdf file and get the NIRS and Event streams
    nirsStream, eventStream = get_NIRS_and_Event_streams(xdf_fullFile)
    # print the name and type of each retruned stream
    print("File: {}".format(xdf_fullFile))
    print(
        "Nirs : {}, {}".format(
            nirsStream["info"]["name"][0], nirsStream["info"]["type"][0]
        )
    )
    print(
        "Event: {}, {}".format(
            eventStream["info"]["name"][0], eventStream["info"]["type"][0]
        )
    )

##     Explore the sequence of markers in the Event stream and plot the data and markers 

In [ ]:
def explore_xdf_markers_sequence(nirsStream, eventStream):
    """
    Explore the sequence of markers in the Event stream
    """

    nirsData_xdf = nirsStream["time_series"]
    nirsTime_xdf = nirsStream["time_stamps"]
    eventData_xdf = eventStream["time_series"]
    eventTime_xdf = eventStream["time_stamps"]

    # find the set of possible events using np.unique
    events = np.unique(eventData_xdf)
    print("Found {} events with labels in {}".format(len(eventData_xdf), events))

    # find all events containing the word 111 and 100
    i111 = []
    i100 = []
    for i in range(len(eventData_xdf)):
        if "111" in eventData_xdf[i][0]:
            i111.append(i)
        if "100" in eventData_xdf[i][0]:
            i100.append(i)
    print("Found {} events 111".format(len(i111)))
    print("Found {} events 100".format(len(i100)))

    iStart = i111[0]

    print(
        "First event 111 is at index {} with label {} at {:10.3f}s".format(
            iStart,
            eventData_xdf[iStart][0],
            eventTime_xdf[iStart],
        ),
    )

    # get the labels for the NIRS data
    nirsLabels = []
    for chan in nirsStream["info"]["desc"][0]["channels"][0]["channel"]:
        label = chan["label"]
        unit = chan["unit"]
        type = chan["type"]
        nirsLabels.append({"label": label, "unit": unit, "type": type})
    print("Found {} channels: ".format(len(nirsLabels)))
    for i in range(len(nirsLabels)):
        print(
            "  {:02d}: {} ({} {})".format(
                i,
                nirsLabels[i]["label"][0][8:],  # remove the first 8 characters
                nirsLabels[i]["type"][0],
                nirsLabels[i]["unit"][0],
            )
        )

    # keep only the NIRS data channels with type NIRS and unit OD
    nirs = []
    labels = []
    for i in range(len(nirsLabels)):
        if nirsLabels[i]["type"][0] == "NIRS" and nirsLabels[i]["unit"][0] == "OD":
            nirs.append(nirsData_xdf[:, i])
            labels.append(nirsLabels[i]["label"][0][8:])

    # make boxplot of all channels in the NIRS data, with labels in nirsLabels
    plt.figure()
    plt.boxplot(nirs)
    plt.xticks(range(1, len(labels) + 1), labels)
    # write labels vertically
    plt.xticks(rotation=90)
    plt.title("All NIRS channels")
    plt.show()

    # keep only the channels that have a median lower than 3
    nirsActive = []
    labelsActive = []
    for i in range(len(nirs)):
        if np.median(nirs[i]) < 4.5:
            nirsActive.append(nirs[i])
            labelsActive.append(labels[i])

    # make boxplot of all channels in the **real** NIRS data channels
    plt.figure()
    plt.boxplot(nirsActive)
    plt.xticks(range(1, len(labelsActive) + 1), labelsActive)
    # write labels vertically
    plt.xticks(rotation=90)
    plt.title("NIRS channels with median < 4.5")
    plt.show()

    # plot the NIRS data with vertical lines at the event times
    plt.figure()

    myTime_x = nirsTime_xdf - eventTime_xdf[iStart]
    myTime_x_event = eventTime_xdf - eventTime_xdf[iStart]

    # plot nirs as a function of time
    for i in range(len(nirsActive)):
        plt.plot(myTime_x, nirsActive[i], label=labelsActive[i])

    for i in range(len(eventData_xdf)):
        e = eventData_xdf[i][0]
        t = myTime_x_event[i]
        plt.axvline(x=t, color="lightgray")
        if "100" in e:
            plt.axvline(x=t, color="green")
        elif "111" in e:
            plt.axvline(x=t, color="red")

    plt.xlabel("Time (s)")
    plt.ylabel("OD")
    # put the legend outside the plot
    ax = plt.gca()
    # Shrink current axis by 20%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.9, box.height])

    # Put a legend to the right of the current axis
    ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

    plt.title("NIRS data with events")
    plt.show()

    return nirsActive, labelsActive


if doRunTests:
    nirsStream, eventStream = get_NIRS_and_Event_streams(xdf_fullFile)
    nirsActive, labelsActive = explore_xdf_markers_sequence(nirsStream, eventStream)

# Read the snirf file

In [ ]:
# Load the snirf file
import mne

s_file = "/Users/denismottet/Documents/GitHub/NeuArm-DataAnalysis/data/ReArm.lnk/twoTestPatientsForOXY4/C1P07_20210802_1_r.snirf"

raw = mne.io.read_raw_snirf(s_file, preload=True, verbose="CRITICAL")
raw_data = raw.get_data()
print(raw_data.shape)
# plot the raw data (to see the markers-annotations)
# raw.plot()


# get the annotations (there is no Events in this file)
annotations = raw.annotations

# print all annotations
for i in range(len(annotations.description)):
    e = annotations.description[i]
    t = annotations.onset[i]
    # print("{:2d}: {:10.3f}s = {} ".format(i, t, e))

# find the first annotation containing the word 111
iStart = -1
for i in range(len(annotations.description)):
    if "111" in annotations.description[i]:
        iStart = i
        break
print(
    "First event 111 is at index {} with label {} at time {}".format(
        iStart, annotations.description[iStart], annotations.onset[iStart]
    ),
)

# find all annotations containing the word 111
i111s = []
for i in range(len(annotations.description)):
    if "111" in annotations.description[i]:
        i111s.append(i)
print("All events 111 are at indexes {}".format(i111s))

# Validation of the snirf file

In [ ]:
from snirf import validateSnirf
result = validateSnirf(s_file)
result.display(severity=3)

# find the xdf maker sequence in the snirf markers sequence

In [ ]:
def find_index_of_xdf_markers_in_snirf_annotations(eventStream, annotations):
    """
    Find the index of the xdf markers sequence that is also in the snirf annotations
    """
    iStart = None
    eventData_xdf = eventStream["time_series"]
    eventTime_xdf = eventStream["time_stamps"]
    xdf_sequence = np.diff(eventTime_xdf)
    for i in range(len(xdf_sequence)):
        xdf_beg = i
        xdf_end = i + len(xdf_sequence) + 1
        if xdf_end < len(annotations.onset):
            snirf_sequence = np.diff(annotations.onset[xdf_beg:xdf_end])
            if np.allclose(xdf_sequence, snirf_sequence, atol=0.1):
                # ensure that the labels are with the same number
                same_labels = True
                for j in range(len(eventData_xdf)):
                    # check the last 3 characters (the number)
                    label_xdf = eventData_xdf[j][0][-3:]
                    label_snirf = annotations.description[xdf_beg + j][-3:]
                    if label_xdf != label_snirf:
                        print("labels are not the same")
                        print("xdf label = {}".format(eventData_xdf[j][0]))
                        print(
                            "snirf label = {}".format(
                                annotations.description[xdf_beg + j]
                            )
                        )
                        same_labels = False
                        break
                if same_labels:
                    iStart = xdf_beg
                    break
    return iStart, xdf_sequence, snirf_sequence


if doRunTests:
    (
        iStart,
        xdf_sequence,
        snirf_sequence,
    ) = find_index_of_xdf_markers_in_snirf_annotations(eventStream, annotations)
    if iStart is None:
        print("No corresponding sequence found in snirf")
    else:
        print(
            "Corresponding sequence found at index {} of markers in snirf".format(
                iStart
            )
        )

    difference = xdf_sequence - snirf_sequence
    # create a figure with the plot of the difference (left panel) and a boxplot of the difference (right panel)

    # create a grid with 1 row and 2 columns of different width
    f, (a0, a1) = plt.subplots(1, 2, width_ratios=[10, 1])

    # plot the difference as dots
    a0.plot(difference * 1000, "o")
    a0.set_ylabel("Difference (xdf -snirf) in milliseconds")
    a0.set_xlabel("Index")
    a0.set_title("Difference (xdf -snirf) in milliseconds")

    # make a boxplot without box o
    a1.boxplot(difference * 1000, vert=True, widths=0.8)
    a1.set_axis_off()

    plt.show()

# restrict the snirf data to the same time as the xdf data


In [ ]:
def restrict_snirf_to_xdf_time(raw, annotations, iStart, eventTime_xdf, nirsTime_xdf):
    """
    Restrict the snirf data to the same time as the xdf data
    """
    # get the time of the first and last event in the xdf data

    duration_xdf = nirsTime_xdf[-1] - nirsTime_xdf[0]

    time_first_event_xdf = eventTime_xdf[0] - 0.2
    delay_first_event_xdf = time_first_event_xdf - nirsTime_xdf[0] 
    tmin = annotations.onset[iStart] - delay_first_event_xdf
    # tmax = annotations.onset[iStart + len(eventData_xdf)]
    tmax = tmin + duration_xdf

    # restrict the snirf data to the same time as the xdf data
    raw_new = mne.io.Raw.copy(raw)
    raw_new.crop(tmin=tmin, tmax=tmax)

    return raw_new, tmin


eventData_xdf = eventStream["time_series"]
eventTime_xdf = eventStream["time_stamps"]
nirsTime_xdf = nirsStream["time_stamps"]


tmin = annotations.onset[iStart]
tmax = annotations.onset[iStart + len(eventData_xdf)]
duration_xdf = nirsTime_xdf[-1] - nirsTime_xdf[0]
tmax = annotations.onset[iStart] + duration_xdf


duration_xdf = nirsTime_xdf[-1] - nirsTime_xdf[0]
print("duration xdf = {}".format(duration_xdf))

raw_new, time_zero = restrict_snirf_to_xdf_time(
    raw, annotations, iStart, eventTime_xdf, nirsTime_xdf
)


snirf_labels = raw_new.ch_names
snirf_time = raw_new.times
snirf_data = raw_new.get_data().T
snirf_markers = raw_new.annotations.description
snirf_markers_time = raw_new.annotations.onset
print(snirf_data.shape)
print("data {} {}".format(snirf_time[0], snirf_time[-1]))
print("mark {} {}".format(eventTime_xdf[0], eventTime_xdf[-1]))

xdf_labels = np.array(labelsActive)
xdf_data = np.array(nirsActive).T
xdf_time = np.array(nirsTime_xdf)
xdf_markers = np.array(eventData_xdf)
xdf_markers_time = np.array(eventTime_xdf)

print(xdf_data.shape)

# Re-order the xdf channels to match the snirf channels 

In [ ]:
# reorder the labels = put 757 first, then 852 
# order is unchanged within the 757 and 852 groups
xdf_labels_reordered = []
xdf_data_reordered = []
for i in range(len(xdf_labels)):
    label = xdf_labels[i]
    if "757" in label:
        xdf_labels_reordered.append(label)
        xdf_data_reordered.append(xdf_data[:, i])
for i in range(len(xdf_labels)):
    label = xdf_labels[i]
    if "852" in label:
        xdf_labels_reordered.append(label)
        xdf_data_reordered.append(xdf_data[:, i])

for i in range(len(xdf_labels_reordered)):
    print("{:02d}: {}".format(i, xdf_labels_reordered[i]))

xdf_labels = np.array(xdf_labels_reordered)
xdf_data = np.array(xdf_data_reordered).T

# Plot the snirf and xdf data

In [ ]:
# # diff the snirf data
# snirf_data = np.diff(snirf_data, axis=0)
# xdf_data = np.diff(xdf_data, axis=0)
# # add a leading zero to the snirf data
# snirf_data = np.concatenate((np.zeros((1, snirf_data.shape[1])), snirf_data), axis=0)
# xdf_data = np.concatenate((np.zeros((1, xdf_data.shape[1])), xdf_data), axis=0)
# xdf_data = - xdf_data

# # restrict snirf to channel 9 and xdf to channel 2
# snirf_selected = 9
# xdf_selected = 2
# snirf_data = snirf_data[:, snirf_selected:snirf_selected+1]
# snirf_labels = snirf_labels[snirf_selected:snirf_selected+1]
# xdf_data = xdf_data[:, xdf_selected:xdf_selected+1]
# xdf_labels = labelsActive[xdf_selected:xdf_selected+1]


plt.figure()
# plt.plot(snirf_time, snirf_data)

# plot nirs as a function of time
for i in range(len(snirf_labels)):
    plt.plot(snirf_time, snirf_data[:, i], label=snirf_labels[i])

# add a vertical line at the time of each event
for i in range(len(snirf_markers_time)):
    e = snirf_markers[i]
    t = snirf_markers_time[i] - time_zero
    plt.axvline(x=t, color="lightgray")
    if "100" in e:
        plt.axvline(x=t, color="green")
    elif "111" in e:
        plt.axvline(x=t, color="red")
plt.xlabel("Time (s)")
plt.title("snirf data")
# put the legend outside the plot
ax = plt.gca()
# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.9, box.height])
# Put a legend to the right of the current axis
ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

plt.show()

# check the previous plot with mne
# raw_new.plot(duration=duration_xdf, start=0, scalings="auto")


plt.figure()

# plot nirs as a function of time
for i in range(len(xdf_labels)):
    plt.plot(xdf_time, xdf_data[:, i], label=xdf_labels[i])
# plt.plot(xdf_time, xdf_data)

# add a vertical line at the time of each event
for i in range(len(xdf_markers_time)):
    e = xdf_markers[i][0]
    t = xdf_markers_time[i]
    plt.axvline(x=t, color="lightgray")
    if "100" in e:
        plt.axvline(x=t, color="green")
    elif "111" in e:
        plt.axvline(x=t, color="red")
plt.xlabel("Time (s)")
plt.title("xdf data")
# put the legend outside the plot
ax = plt.gca()
# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.9, box.height])
# Put a legend to the right of the current axis
ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

plt.show()

# Plot ONE snirf and xdf signal on the same plot

This plot shows that the snirf and xdf data are similar, but :
- the zero (by channel) is different for the two data sets
- the amplitude (by channel) is different for the two data sets
- the sign is reversed for the two data sets

In [ ]:
import copy 

# define a common time axis
xdf_time = xdf_time - xdf_time[0]
snirf_time = snirf_time - snirf_time[0]




for ch in range(len(xdf_labels)):

    xdf_chanel = [ch]
    snirf_channel = [ch]

    x_dat = copy.deepcopy(xdf_data[:, xdf_chanel[i]])
    s_dat = copy.deepcopy(snirf_data[:, snirf_channel[i]])

    # define a common y axis zero
    #print("mean = x{:5.4f} s{:5.4f} {:5.4f}".format(np.mean(x_dat, axis=0), np.mean(s_dat, axis=0), np.mean(x_dat, axis=0) - np.mean(s_dat, axis=0)))  
    x_dat = x_dat - np.mean(x_dat, axis=0)
    s_dat = s_dat - np.mean(s_dat, axis=0)

    # define a common y axis scale
    print("std  = x{:5.4f} s{:5.4f} {:5.4f}".format(np.std(x_dat, axis=0), np.std(s_dat, axis=0), np.std(x_dat, axis=0) / np.std(s_dat, axis=0)))
    x_dat = x_dat / np.std(x_dat, axis=0)
    s_dat = s_dat / np.std(s_dat, axis=0)

    # # define a common y axis orientation only for the odd channels
    # if ch % 2 == 1:
    x_dat = -x_dat

    plt.figure()
    # plot nirs as a function of time
    for i in range(len(xdf_chanel)):
        plt.plot(xdf_time, x_dat, label=xdf_labels[xdf_chanel[i]])
    for i in range(len(snirf_channel)):
        plt.plot(snirf_time, s_dat, label=snirf_labels[snirf_channel[i]])
        # with the same time axis
        #plt.plot(xdf_time, snirf_data[:, snirf_channel[i]], label=snirf_labels[snirf_channel[i]])
    
    plt.xlabel("Time (s)")
    plt.ylabel("OD")
    plt.title("xdf + snirf Channel {}".format(ch))
    plt.legend()
    plt.show()


# Correction of the snirf data to match the xdf data

The Oxysoft software (Artinis) transforms the raw data to optical density (OD) and then to hemoglobin concentration (Hb). The transformation is done by the following formula:

$$
Hb = \frac{OD}{\epsilon} \times \frac{1}{d}
$$

where $\epsilon$ is the extinction coefficient and $d$ is the distance between the source and detector.

The Oxysoft software uses the following values for the extinction coefficient:

$$
\epsilon_{HbO2} = 0.0001 \\
\epsilon_{Hb} = 0.0001

$$

The piece of matlab code used to transform the data is the following:

```matlab
        data.dataTimeSeries = 1./exp(log(10).* [rawvals(:, 2:2:end) rawvals(:, 1:2:end)]); %change dataTimeSeries to correct values
``` 

The reverse transformation is done by the following python code:

```python
        data.dataTimeSeries = 1./np.exp(np.log(10)*np.concatenate((rawvals[:, 1::2], rawvals[:, 0::2]), axis=1))
```



In [ ]:
# 

In [ ]:
# new figure
plt.figure()

myTime_s = raw.times - raw.annotations.onset[0]
myTime_s_event = annotations.onset - annotations.onset[iStart]

myTime_x = nirsTime_xdf - eventTime_xdf[iStart]
myTime_x_event = eventTime_xdf - eventTime_xdf[iStart]

# subplot 1
plt.subplot(2, 1, 1)

# plot the signal
for i in range(1):  # raw.get_data().shape[0]):
    toPlot = raw.get_data()[i, :]
    # toPlot = np.append(np.diff(toPlot), 0)
    plt.plot(myTime_s, toPlot)

# toPlot_x = nirsData[:, 1]
# toPlot_x = -(toPlot_x - np.mean(toPlot_x)) +  np.mean(toPlot_x)
# toPlot_x = np.append(np.diff(toPlot_x), 0)
# plt.plot(myTime_x, toPlot_x, color="black")

plt.axvline(x=0, color="red")


# plot the anotations
for i in range(len(annotations.description)):
    e = annotations.description[i]
    t = myTime_s_event[i]
    plt.axvline(x=t, color="lightgray")
    if "100" in e:
        plt.axvline(x=t, color="green", linestyle="dashed")
    elif "111" in e:
        plt.axvline(x=t, color="red", linestyle="dashed")


plt.subplot(2, 1, 2)

# plot the signal
for i in range(1):  # , raw.get_data().shape[0]):
    toPlot = raw.get_data()[i, :]
    # toPlot = np.append(np.diff(toPlot), 0)
    plt.plot(myTime_s, toPlot)

# plot the events
for i in range(len(eventData_xdf)):
    e = eventData_xdf[i][0]
    t = myTime_x_event[i]
    plt.axvline(x=t, color="lightgray")
    if "100" in e:
        plt.axvline(x=t, color="green")
    elif "111" in e:
        plt.axvline(x=t, color="red")

plt.show()